In [ ]:
!pip install langchain qdrant-client llama-cpp-python pandas transformers sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_huggingface

In [ ]:
import os
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from huggingface_hub import login
from langchain.vectorstores import Qdrant
from langchain.embeddings import LlamaCppEmbeddings
from qdrant_client import QdrantClient
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


# Hugging Face login token
huggingface_token = "HUGGINGFACE_TOKEN"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#meta-llama/Meta-Llama-3.1-8B-Instruct
#meta-llama/Llama-2-7b-chat-hf
# Qdrant configuration
url = "QDRANT_URL"
api_key ="QDRANT_API_KEY"
collection_name ="COLLECTION_NAME"


# Se connecter à Hugging Face avec le token
login(token=huggingface_token)
use_4bit = True
compute_dtype = getattr(torch, "float16")
device_map = {"": 0}


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:

# Load the model and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [2]:
import pandas as pd

from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from langchain.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings
# Load your dataset
df = pd.read_excel('dataset_agriculture_18K.xlsx')
df


,Prompt,Context,Prompt_context
0,Prunier: Taille de formation,Il faut intervenir sur les pruniers les 7 prem...,Prunier: Taille de formation \n Il faut interv...
1,Quels outils utiliser pour la taille d'un prun...,"Pour effectuer la taille d'un prunier, il faut...",Quels outils utiliser pour la taille d'un prun...
2,Comment procéder à la taille d'un prunier ?,"Pour tailler un prunier efficacement, il suffi...",Comment procéder à la taille d'un prunier ? \n...
3,Semis : comment bien choisir son noyau de pêche ?,"Avant de planter un noyau, choisir une pêche d...",Semis : comment bien choisir son noyau de pêch...
4,Pourquoi installent-on des pêchers dans les vi...,Les pêches mûrissent au moment des vendanges. ...,Pourquoi installent-on des pêchers dans les vi...
...,...,...,...
18204,Pourquoi choisir une zone large et ensoleillée...,Il est important de choisir un endroit large a...,Pourquoi choisir une zone large et ensoleillée...
18205,Quel est le sol approprié pour la culture des ...,Si votre sol a été utilisé pour cultiver des f...,Quel est le sol approprié pour la culture des ...
18206,Comment protéger les plants de citrouilles des...,Il est important de choisir un endroit pour pl...,Comment protéger les plants de citrouilles des...
18207,Quel est le meilleur moment pour planter des c...,Les citrouilles sont très sensibles aux gelées...,Quel est le meilleur moment pour planter des c...


In [ ]:


texts = df['Prompt_context'].tolist()  # Replace 'text_column_name' with the actual column name containing the text

vectors_config = VectorParams(size=384, distance=Distance.COSINE)
qdrant_client = QdrantClient(url=url, api_key=api_key)
# Supprimer la collection existante si elle existe
if qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.delete_collection(collection_name=collection_name)

# Créer une nouvelle collection
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=vectors_config
)
# Create LangChain Qdrant instance with HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
qdrant = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=hf_embeddings
)

# Add the texts to the Qdrant collection, embeddings will be calculated by HuggingFaceEmbeddings
qdrant.add_texts(texts=texts)

print("Data has been added to the Qdrant collection successfully.")



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


Data has been added to the Qdrant collection successfully.


In [ ]:
sysprompt="""Vous êtes un assistant chargé de répondre aux questions des agriculteurs marocains.
On vous donne des extraits de documents ainsi qu'une question. Fournissez une réponse conversationnelle et précise.
La réponse doit être dans la même langue que la question posée par l'utilisateur.
Si les documents fournis ne sont pas pertinents pour la question, ne les utilisez pas."""


In [ ]:
from langchain.schema import Document  #
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts.prompt import PromptTemplate
from transformers import pipeline #Import pipeline

template = f"""
{sysprompt}

Question: {{question}}

Documents:
{{documents}}

Réponse:
"""

# Load QA chain
qa_prompt = PromptTemplate(template=template, input_variables=["documents", "question"])  # Use the correct input variables

# Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    return_full_text=True,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.4,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.15
)
# Wrap the pipeline in a LangChain HuggingFacePipeline object
llm = HuggingFacePipeline(pipeline=pipe)
# Load the QA chain with the llm object
chain = load_qa_chain(
    llm=llm,
    chain_type="stuff",
    prompt=qa_prompt,  # Use qa_prompt here
    document_variable_name="documents"  # Tell the chain to use "documents" for the docs
)


In [ ]:

# Requête utilisateur
user_query = "Comment puis-je améliorer la récolte de mes olives?"
# Génération de l'embedding pour la requête
query_embedding = hf_embeddings.embed_query(user_query)
# Recherche dans Qdrant pour récupérer les documents les plus pertinents
retrieved_docs = qdrant.similarity_search(query_embedding, top_k=3)
# Génération de la réponse en utilisant les documents récupérés
response = qa_chain.run(question=user_query, documents=[doc.page_content for doc in retrieved_docs])
print("Réponse générée : ", response)


In [ ]:
# Générer la réponse avec les documents récupérés
# Assuming 'retrieved_docs' contains QdrantDocument objects, extract the page_content
retrieved_docs = [Document(page_content=doc.page_content) for doc in retrieved_docs]  # Extract page_content from QdrantDocument

response = chain.run(question=user_query, input_documents=retrieved_docs)  # Use input_documents instead of context

print("Réponse générée : ", response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Réponse générée :  
Vous êtes un assistant chargé de répondre aux questions des agriculteurs marocains.
On vous donne des extraits de documents ainsi qu'une question. Fournissez une réponse conversationnelle et précise.
La réponse doit être dans la même langue que la question posée par l'utilisateur.
Si les documents fournis ne sont pas pertinents pour la question, ne les utilisez pas.

Question: Comment puis-je améliorer la récolte de mes olives?

Documents:
Quels produits peuvent faciliter le détachement des olives et comment fonctionnent-ils ? 
 Des produits favorisant l'abscission peuvent accélérer la maturation des olives et faciliter leur détachement en réduisant la résistance entre le fruit et le pédoncule. Leur efficacité peut varier et est généralement plus utile pour les récoltes au sol en accélérant la chute naturelle des fruits.

Quelles sont les méthodes de récolte des olives et dans quels cas sont-elles utilisées ? 
 Les méthodes de récolte des olives varient : la récolte

In [ ]:
answer=response.split("Réponse:")[1].strip()

In [ ]:
answer

"Bonjour! Pour améliorer la récolte de vos olives, je vous recommande de considérer plusieurs facteurs. Tout d'abord, assurez-vous que votre arbre bénéficie d'un irrigation adéquat. Les oliviers ont besoin de 2,5 cm d'eau par semaine, mais il est important de noter que certaines périodes peuvent requérir moins d'eau. Ensuite, assurez-vous que vos olives soient matures suffisamment avant de les récolter. Si vous avez des olives encore vertes, il est possible qu'elles ne soient pas prêtes à être récoltées.\n\nEnsuite, si vous avez des problèmes avec le détachement des olives, vous pouvez essayer utiliser des produits favorisant l'abscission. Ces produits peuvent aider à réduire la résistance entre le fruit et le pédoncule, facilitant ainsi le détachement naturel des olives.\n\nIl est également important de choisir la méthode de récolte appropriée pour vos besoins. Si vous voulez obtenir des olives de haute qualité, la récolte à la main pourrait être la meilleure option. Sinon, vous pouve

In [ ]:
def generate(user_query):
  # Générer l'embedding pour la requête
    query_embedding = hf_embeddings.embed_query(user_query)
  # Recherche dans Qdrant pour récupérer les documents pertinents
  
    retrieved_docs = qdrant.similarity_search(user_query, k=5)
  # Générer la réponse avec les documents récupérés
  # Assuming 'retrieved_docs' contains QdrantDocument objects, extract the page_content
    retrieved_docs = [Document(page_content=doc.page_content) for doc in retrieved_docs]  # Extract page_content from QdrantDocument
    response = chain.run(question=user_query, input_documents=retrieved_docs)  # Use input_documents instead of context
    answer=response.split("Réponse:")[1].strip()
    print("Réponse générée : ", response)
    return answer

In [ ]:
import pandas as pd

df = pd.read_excel('df_test.xlsx')
print(len(df))

def generate_responses(df):
    responses = []
    responses1 = []
    for prompt in df['question']:
        print(prompt)

        response1 = generate(prompt)
        responses1.append(response1)

    df['generated_response_5'] = responses1

    return df

df = generate_responses(df)

In [ ]:
df.to_excel('df_test_qdrant_llama3.xlsx')

In [4]:
!pip install rouge-score # Install the rouge-score module.
import pandas as pd
import torch
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Download NLTK resources
nltk.download('punkt')

# Initialize the GPT-2 model and tokenizer
model_name = 'gpt2'
modelgpt = GPT2LMHeadModel.from_pretrained(model_name)
tokenizergpt = GPT2Tokenizer.from_pretrained(model_name)

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate perplexity
def calculate_perplexity(text):
    inputs = tokenizergpt(text, return_tensors='pt', truncation=True, max_length=1024)
    outputs = modelgpt(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return torch.exp(loss).item()

# Function to calculate BLEU, ROUGE, and perplexity scores
def calculate_scores(df):
    bleu_scores = []
    rouge_scores_1 = []
    rouge_scores_2 = []
    rouge_scores_l = []
    perplexities = []

    for idx, row in df.iterrows():
        context = row['answer']
        response = row['Generated Responses']

        # BLEU score with smoothing function
        context_tokens = nltk.word_tokenize(context)
        response_tokens = nltk.word_tokenize(response)
        bleu = sentence_bleu([context_tokens], response_tokens, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu)

        # ROUGE scores
        rouge_score = scorer.score(context, response)
        rouge_scores_1.append(rouge_score['rouge1'].fmeasure)
        rouge_scores_2.append(rouge_score['rouge2'].fmeasure)
        rouge_scores_l.append(rouge_score['rougeL'].fmeasure)

        # Perplexity
        perplexity = calculate_perplexity(response)
        perplexities.append(perplexity)

    # Add scores to DataFrame
    df['BLEU'] = bleu_scores
    df['ROUGE-1'] = rouge_scores_1
    df['ROUGE-2'] = rouge_scores_2
    df['ROUGE-L'] = rouge_scores_l
    df['Perplexity'] = perplexities

    return df


# Calculate scores
df = calculate_scores(df)

# Calculate global scores
mean_bleu = df['BLEU'].mean()
mean_rouge_1 = df['ROUGE-1'].mean()
mean_rouge_2 = df['ROUGE-2'].mean()
mean_rouge_l = df['ROUGE-L'].mean()
mean_perplexity = df['Perplexity'].mean()

print("Scores globaux:")
print(f"BLEU: {mean_bleu}")
print(f"ROUGE-1: {mean_rouge_1}")
print(f"ROUGE-2: {mean_rouge_2}")
print(f"ROUGE-L: {mean_rouge_l}")
print(f"Perplexity: {mean_perplexity}")

# Save results to CSV
df.to_csv('evaluation_results_qdrant_LANGCHAIN.csv', index=False)



  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5a07eb4cc0684ceba9868e0f8aa92b9cd5f7572a1d2f9f0ef4ec89ad0b110167
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]
model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]
generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]
tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]
vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]
merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]
tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]
Scores globaux:
BLEU: 0.056523650569383276
ROUGE-1: 0.35767703405428158
ROUGE-2: 0.12051696559958655
ROUGE-L: 0.20067981006505091
Perp